## **4. Construcción de Modelos** (entrenamiento)

In [ ]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import r2_score, mean_squared_error, accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import xgboost as xgb
import lightgbm as lgb
# Si se usa colab hay que instalar catboost
try:
    import catboost as cat
except Exception:
    !pip install catboost
finally:
    import catboost as cat

from datetime import date
from joblib import dump, load

from sklearn.decomposition import PCA
import plotly.express as px
from sklearn.calibration import CalibratedClassifierCV
import seaborn as sns
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import VotingClassifier, VotingRegressor

import warnings; warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [2]:
# CARGA DE DATOS POST ANALISIS
df = pd.read_csv('datos_luego_de_eda.csv')

---

### Modelo XGBoost

In [ ]:
data = df.copy()

In [ ]:
def xgbclass(seed=42, data=data):
        # Clasificador
        clf_xgb = xgb.XGBClassifier(objective='binary:logistic', random_state=seed, n_jobs=-1)

        # Otra forma de hacerlo, con clasificador y regresor usando xgboost
        X = data.drop('transactionRevenue', axis=1)
        y = data['transactionRevenue'].apply(lambda x: 1 if x > 0 else 0)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

        # Creamos un random search para encontrar los mejores hiperparámetros
        params = {
                'booster': ['gbtree', 'gblinear', 'dart'], # Tipo de modelo
                'n_estimators': [100, 200, 400], # Número de árboles
                'learning_rate': [0.1, 0.06, 0.03, 0.01],  # Tasa de aprendizaje
                }

        # Obtenemos los mejores hiperparámetros
        search = RandomizedSearchCV(
                estimator=clf_xgb, 
                param_distributions=params, 
                scoring='accuracy',
                cv=5, 
                n_iter=5)
        search.fit(X_train, y_train)
        best_params = search.best_params_

        # Entrenamos el clasificador
        clf_xgb = xgb.XGBClassifier(**best_params, random_state=seed, n_jobs=-1)
        clf_xgb.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=10, verbose=0)

        # Predecimos con el clasificador
        y_pred_cls_xgb = clf_xgb.predict(X_test)

        # Score del clasificador
        print(f'Precisión del clasificador: {accuracy_score(y_test, y_pred_cls_xgb):.3f}')

        # Calibración más Precisa de las Probabilidades
        cal_clf_xgb = CalibratedClassifierCV(clf_xgb, method='sigmoid')
        cal_clf_xgb.fit(X_train, y_train)

        # Probabilidades de Clase Calibradas
        calibrated_y_pred = cal_clf_xgb.predict(X_test)

        # Medir el Rendimiento
        print(f'Precisión del clasificador calibrado: {accuracy_score(y_test, calibrated_y_pred):.3f}')

        if accuracy_score(y_test, y_pred_cls_xgb) < accuracy_score(y_test, calibrated_y_pred):
            clf_xgb = cal_clf_xgb

        return clf_xgb

In [ ]:
clf_xgb = xgbclass()

In [ ]:
def xgbreg(seed=42, data=data):
        # Regresor
        reg_xgb = xgb.XGBRegressor(objective='reg:squarederror', random_state=seed)

        # Agregamos la columna 'class' y dividimos en entrenamiento y prueba
        data['class'] = clf_xgb.predict(data.drop(columns=['transactionRevenue']))
        X = data.drop(columns=['transactionRevenue'])
        y = data['transactionRevenue']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

        # Creamos un random search para encontrar los mejores hiperparámetros
        params = {
                'booster': ['gbtree', 'gblinear', 'dart'], # Tipo de modelo
                'n_estimators': [50, 150, 500], # Número de árboles
                'learning_rate': [0.1, 0.06, 0.03, 0.01],  # Tasa de aprendizaje
                }

        # Obtenemos los mejores hiperparámetros
        search = RandomizedSearchCV(
                estimator=reg_xgb, 
                param_distributions=params, 
                scoring='neg_mean_squared_error', 
                cv=5, 
                n_iter=5)
        search.fit(X_train, y_train)
        best_params = search.best_params_

        # Entrenamos el regresor agregando los mejores hiperparámetros
        reg_xgb = xgb.XGBRegressor(**best_params, objective='reg:squarederror', random_state=seed)
        reg_xgb.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=10, verbose=0)

        # Predecimos con el regresor
        y_pred_reg_xgb = reg_xgb.predict(X_test)

        # Convertimos todas las predicciones menores a 0.01 en 0
        y_pred_reg_xgb = np.where(y_pred_reg_xgb < 0.01, 0, y_pred_reg_xgb)

        # Score del regresor para rmse y r2
        print(f'Raiz del error para el regresor: {np.sqrt(mean_squared_error(y_test, y_pred_reg_xgb)):.3f}')

        return reg_xgb, y_pred_reg_xgb, np.sqrt(mean_squared_error(y_test, y_pred_reg_xgb)), r2_score(y_test, y_pred_reg_xgb)

In [ ]:
rmse = 21
while rmse >= 16:
    data = df.copy()
    seed = np.random.randint(1, 100000)
    reg_xgb, reg_xgb_pred, xgb_rmse, xgb_r2 = xgbreg(seed, data)
    if rmse < 16:
        print('El regresor logro el objetivo:')
        print(f'El valor de la semilla es: {seed}')
        print(f'Los hiperparámetros son: {reg_xgb.get_params()}')
        print(f'El rmse es: {xgb_rmse}')
        print(f'El r2 es: {xgb_r2}')

> mejores hiperparametros encontrados: ``{'objective': 'reg:squarederror', 'base_score': None, 'booster': 'gblinear', 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': None, 'feature_types': None, 'gamma': None, 'gpu_id': None, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': 0.1, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': None, 'max_leaves': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'n_estimators': 500, 'n_jobs': None, 'num_parallel_tree': None, 'predictor': None, 'random_state': 31510, 'reg_alpha': None, 'reg_lambda': None, 'sampling_method': None, 'scale_pos_weight': None, 'subsample': None, 'tree_method': None, 'validate_parameters': None, 'verbosity': None}``

In [ ]:
# # Guardamos el modelo XGBoost
# dump(clf_xgb, 'clasificador_xgb.joblib')
# dump(reg_xgb, 'regresor_xgb.joblib')

### Modelo LGBM

In [ ]:
data = df.copy()

In [ ]:
def lbgclass(seed=42, data=data):
        # Clasificador
        clf_lgb = lgb.LGBMClassifier(random_state=seed, n_jobs=-1, verbosity=-1)

        # Dividimos en entrenamiento y prueba
        X = data.drop('transactionRevenue', axis=1)
        y = data['transactionRevenue'].apply(lambda x: 1 if x > 0 else 0)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


        # Creamos un random search para encontrar los mejores hiperparámetros
        params = {
                'boosting_type': ['gbdt', 'dart', 'goss'], # Tipo de modelo
                'num_boost_round': [100, 400, 800], # Número de árboles
                'max_depth': [2, 3, 5], # Profundidad máxima
                'learning_rate': [0.1, 0.06, 0.03, 0.01],  # Tasa de aprendizaje
                }

        # Obtenemos los mejores hiperparámetros
        search = RandomizedSearchCV(
                estimator=clf_lgb, 
                param_distributions=params, 
                scoring='accuracy', 
                cv=5, 
                n_iter=5)
        search.fit(X_train, y_train)
        best_params = search.best_params_

        # Entrenamos el clasificador
        clf_lgb = lgb.LGBMClassifier(**best_params, random_state=seed, n_jobs=-1, verbosity=-1)
        clf_lgb.fit(X_train, y_train, eval_set=[(X_test, y_test)])

        # Predecimos con el clasificador
        y_pred_cls_lgb = clf_lgb.predict(X_test)

        # Score del clasificador
        print(f'Precisión del clasificador: {accuracy_score(y_test, y_pred_cls_lgb):.3f}')

        # Calibración más Precisa de las Probabilidades
        cal_clf_lgb = CalibratedClassifierCV(clf_lgb, method='sigmoid')
        cal_clf_lgb.fit(X_train, y_train)

        # Probabilidades de Clase Calibradas
        calibrated_y_pred = cal_clf_lgb.predict(X_test)

        # Medir el Rendimiento
        print(f'Precisión del clasificador calibrado: {accuracy_score(y_test, calibrated_y_pred):.3f}')

        if accuracy_score(y_test, y_pred_cls_lgb) < accuracy_score(y_test, calibrated_y_pred):
            clf_lgb = cal_clf_lgb

        return clf_lgb

In [ ]:
clf_lgb = lbgclass()

In [ ]:
def lgbreg(seed=42, data=data):
        # Regresor
        reg_lgb = lgb.LGBMRegressor(random_state=seed, n_jobs=-1, verbosity=-1)

        # Agregamos la columna 'class' y dividimos en entrenamiento y prueba
        data['class'] = clf_lgb.predict(data.drop(columns=['transactionRevenue']))
        X = data.drop(columns=['transactionRevenue'])
        y = data['transactionRevenue']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

        # Creamos un random search para encontrar los mejores hiperparámetros
        params = {
                'boosting_type': ['gbdt', 'dart', 'goss'], # Tipo de modelo
                'n_estimators': [100, 400, 800], # Número de árboles
                'max_depth': [2, 3, 5], # Profundidad máxima
                'learning_rate': [0.1, 0.06, 0.03, 0.01],  # Tasa de aprendizaje
                }

        # Obtenemos los mejores hiperparámetros
        search = RandomizedSearchCV(
                estimator=reg_lgb, 
                param_distributions=params, 
                scoring='neg_mean_squared_error', 
                cv=5, 
                n_iter=5)
        search.fit(X_train, y_train)
        best_params = search.best_params_

        # Entrenamos el regresor
        reg_lgb = lgb.LGBMRegressor(**best_params, random_state=seed, n_jobs=-1, verbosity=-1)
        reg_lgb.fit(X_train, y_train, eval_set=[(X_test, y_test)])

        # Predecimos con el regresor
        y_pred_reg_lgb = reg_lgb.predict(X_test)

        # Convertimos todas las predicciones menores a 0.01 en 0
        y_pred_reg_lgb = np.where(y_pred_reg_lgb < 0.01, 0, y_pred_reg_lgb)

        # Score del regresor para rmse
        print(f'Raiz del error para el regresor: {np.sqrt(mean_squared_error(y_test, y_pred_reg_lgb)):.3f}')

        return reg_lgb, y_pred_reg_lgb, np.sqrt(mean_squared_error(y_test, y_pred_reg_lgb)), r2_score(y_test, y_pred_reg_lgb)

In [ ]:
rmse = 21
while rmse >= 16:
    data = df.copy()
    seed = np.random.randint(1, 100000)
    reg_lgb, reg_lgb_pred, lgb_rmse, lgb_r2 = lgbreg(seed, data)
    if rmse < 16:
        print('El regresor logro el objetivo:')
        print(f'El valor de la semilla es: {seed}')
        print(f'Los hiperparámetros son: {reg_lgb.get_params()}')
        print(f'El rmse es: {lgb_rmse}')
        print(f'El r2 es: {lgb_r2}')

> mejores hiperparametros encontrados: `{'boosting_type': 'dart', 'class_weight': None, 'colsample_bytree': 1.0, 'importance_type': 'split', 'learning_rate': 0.06, 'max_depth': 5, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 100, 'n_jobs': -1, 'num_leaves': 31, 'objective': None, 'random_state': 17228, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'subsample': 1.0, 'subsample_for_bin': 200000, 'subsample_freq': 0, 'verbosity': -1}`

In [ ]:
# # Guardamos el modelo LGBM
# dump(clf_lgb, 'clasificador_lgb.joblib')
# dump(reg_lgb, 'regresor_lgb.joblib')

### Modelo CatBoost

In [9]:
data = df.copy()

In [10]:
def catclass(seed=42, data=data):
        # Clasificador
        clf_cat = cat.CatBoostClassifier(random_state=seed, verbose=0)

        # Dividimos en entrenamiento y prueba
        X = data.drop('transactionRevenue', axis=1)
        y = data['transactionRevenue'].apply(lambda x: 1 if x > 0 else 0)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

        # Creamos un random search para encontrar los mejores hiperparámetros
        params = {
                'learning_rate': [0.01, 0.05, 0.1, 0.3], # Tasa de aprendizaje
                'depth': [2, 3, 5], # Profundidad máxima
                'num_boost_round': [100, 400, 800] # Número de árboles
                }

        # Obtenemos los mejores hiperparámetros
        search = RandomizedSearchCV(
                estimator=clf_cat, 
                param_distributions=params, 
                scoring='accuracy', 
                cv=5, 
                n_iter=5)
        search.fit(X_train, y_train)
        best_params = search.best_params_

        # Entrenamos el clasificador
        clf_cat = cat.CatBoostClassifier(**best_params, verbose=0, random_state=seed)
        clf_cat.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=20)

        # Predecimos con el clasificador
        y_pred_cls_lgb = clf_cat.predict(X_test)

        # Score del clasificador
        print(f'Precisión del clasificador: {accuracy_score(y_test, y_pred_cls_lgb):.3f}')

        # Calibración más Precisa de las Probabilidades
        cal_clf_cat = CalibratedClassifierCV(clf_cat, method='sigmoid')
        cal_clf_cat.fit(X_train, y_train)

        # Probabilidades de Clase Calibradas
        calibrated_y_pred = cal_clf_cat.predict(X_test)

        # Medir el Rendimiento
        print(f'Precisión del clasificador calibrado: {accuracy_score(y_test, calibrated_y_pred):.3f}')

        if accuracy_score(y_test, y_pred_cls_lgb) < accuracy_score(y_test, calibrated_y_pred):
            clf_cat = cal_clf_cat

        return clf_cat

In [11]:
clf_cat = catclass()

Precisión del clasificador: 0.987
Precisión del clasificador calibrado: 0.986


In [12]:
def catreg(seed=42, data=data):
        # Regresor
        reg_cat = cat.CatBoostRegressor(random_state=seed, verbose=0)

        # Agregamos la columna 'class' y dividimos en entrenamiento y prueba
        data['class'] = clf_cat.predict(data.drop(columns=['transactionRevenue']))
        X = data.drop(columns=['transactionRevenue'])
        y = data['transactionRevenue']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

        # Creamos un random search para encontrar los mejores hiperparámetros
        params = {
                'learning_rate': [0.01, 0.05, 0.1, 0.3], # Tasa de aprendizaje
                'depth': [2, 3, 5], # Profundidad máxima
                'num_boost_round': [100, 400, 800] # Número de árboles
                }

        # Obtenemos los mejores hiperparámetros
        search = RandomizedSearchCV(
                estimator=reg_cat, 
                param_distributions=params, 
                scoring='neg_mean_squared_error', 
                cv=5, 
                n_iter=5)
        search.fit(X_train, y_train)
        best_params = search.best_params_

        # Entrenamos el regresor
        reg_cat = cat.CatBoostRegressor(**best_params, random_state=seed, verbose=0)
        reg_cat.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=10, verbose=0)

        # Predecimos con el regresor
        y_pred_reg_cat = reg_cat.predict(X_test)

        # Convertimos todas las predicciones menores a 0.01 en 0
        y_pred_reg_cat = np.where(y_pred_reg_cat < 0.01, 0, y_pred_reg_cat)

        # Score del regresor para rmse
        print(f'Raiz del error para el regresor: {np.sqrt(mean_squared_error(y_test, y_pred_reg_cat)):.3f}')

        return reg_cat, y_pred_reg_cat, np.sqrt(mean_squared_error(y_test, y_pred_reg_cat)), r2_score(y_test, y_pred_reg_cat)

In [13]:
rmse = 21
while rmse >= 16:
    data = df.copy()
    seed = np.random.randint(1, 100000)
    reg_cat, reg_cat_pred, cat_rmse, cat_r2 = catreg(seed, data)
    if rmse < 16:
        print('El regresor logro el objetivo:')
        print(f'El valor de la semilla es: {seed}')
        print(f'Los hiperparámetros son: {reg_cat.get_params()}')
        print(f'El rmse es: {cat_rmse}')
        print(f'El r2 es: {cat_r2}')

Raiz del error para el regresor: 17.597
Raiz del error para el regresor: 17.407
Raiz del error para el regresor: 17.506
Raiz del error para el regresor: 17.501
Raiz del error para el regresor: 17.352
Raiz del error para el regresor: 17.502
Raiz del error para el regresor: 17.509
Raiz del error para el regresor: 17.595
Raiz del error para el regresor: 17.253
Raiz del error para el regresor: 17.468
Raiz del error para el regresor: 17.496
Raiz del error para el regresor: 17.264
Raiz del error para el regresor: 17.817
Raiz del error para el regresor: 17.611
Raiz del error para el regresor: 17.913
Raiz del error para el regresor: 17.443
Raiz del error para el regresor: 17.489
Raiz del error para el regresor: 17.467
Raiz del error para el regresor: 17.803
Raiz del error para el regresor: 17.385
Raiz del error para el regresor: 17.450
Raiz del error para el regresor: 17.629
Raiz del error para el regresor: 17.484
Raiz del error para el regresor: 17.572


> mejores Hiperparametros encontrados: ``{'learning_rate': 0.01, 'depth': 3, 'loss_function': 'RMSE', 'verbose': 0, 'num_boost_round': 100, 'random_state': 28160}``

In [ ]:
# # Guardamos el modelo CatBoost
# dump(clf_cat, 'clasificador_cat.joblib')
# dump(reg_cat, 'regresor_cat.joblib')

### Modelo de Votacion

In [ ]:
data = df.copy()

In [ ]:
def votclass(data=data):    
    # Dividimos en entrenamiento y prueba
    X = data.drop('transactionRevenue', axis=1)
    y = data['transactionRevenue'].apply(lambda x: 1 if x > 0 else 0)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

    # Clasificador
    clf = VotingClassifier(estimators=[('xgb', clf_xgb), ('lgb', clf_lgb), ('cat', clf_cat)], voting='soft', n_jobs=-1)

    # Entrenamos el clasificador
    clf.fit(X_train, y_train)

    # Predecimos con el clasificador
    y_pred_cls = clf.predict(X_test)

    # Score del clasificador
    print(f'Precisión del clasificador: {accuracy_score(y_test, y_pred_cls):.3f}')

    return clf

In [ ]:
clf = votclass()

In [ ]:
def votreg(data=data):
    # Regresor
    reg = VotingRegressor(estimators=[('xgb', reg_xgb), ('lgb', reg_lgb), ('cat', reg_cat)], n_jobs=-1)

    # Agrergamos las columnas de clasificación
    data['xgb_class'] = clf_xgb.predict(data.drop(columns=['transactionRevenue']))
    data['lgb_class'] = clf_lgb.predict(data.drop(columns=['transactionRevenue', 'xgb_class']))
    data['cat_class'] = clf_cat.predict(data.drop(columns=['transactionRevenue', 'xgb_class', 'lgb_class']))

    X = data.drop(columns=['transactionRevenue'])
    y = data['transactionRevenue']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

    # Entrenamos el regresor
    reg.fit(X_train, y_train)

    # Predecimos con el regresor
    y_pred_reg = reg.predict(X_test)

    # Convertimos todas las predicciones menores a 0.01 en 0
    y_pred_reg = np.where(y_pred_reg < 0.01, 0, y_pred_reg)

    # Score del regresor para rmse
    print(f'Raiz del error para el regresor: {np.sqrt(mean_squared_error(y_test, y_pred_reg)):.3f}')

    return reg, y_pred_reg, np.sqrt(mean_squared_error(y_test, y_pred_reg)), r2_score(y_test, y_pred_reg)


In [ ]:
rmse = 21
while rmse >= 16:
    data = df.copy()
    reg, y_pred_reg, vot_rmse, vot_r2 = votreg(data)
    if rmse < 16:
        print('El regresor logro el objetivo:')
        print(f'Los hiperparámetros son: {reg.get_params()}')
        print(f'El rmse es: {vot_rmse}')
        print(f'El r2 es: {vot_r2}')

> mejores Hiperparametros encontrados: ``{'estimators': [('xgb', XGBRegressor(base_score=None, booster='gblinear', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=31510, ...)), ('lgb', LGBMRegressor(boosting_type='dart', learning_rate=0.06, max_depth=5, n_jobs=-1,
              random_state=17228, verbosity=-1)), ('cat', <catboost.core.CatBoostRegressor object at 0x000002630DE84F90>)], 'n_jobs': -1, 'verbose': False, 'weights': None, 'xgb': XGBRegressor(base_score=None, booster='gblinear', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=31510, ...), 'lgb': LGBMRegressor(boosting_type='dart', learning_rate=0.06, max_depth=5, n_jobs=-1,
              random_state=17228, verbosity=-1), 'cat': <catboost.core.CatBoostRegressor object at 0x000002630DE84F90>, 'xgb__objective': 'reg:squarederror', 'xgb__base_score': None, 'xgb__booster': 'gblinear', 'xgb__callbacks': None, 'xgb__colsample_bylevel': None, 'xgb__colsample_bynode': None, 'xgb__colsample_bytree': None, 'xgb__early_stopping_rounds': None, 'xgb__enable_categorical': False, 'xgb__eval_metric': None, 'xgb__feature_types': None, 'xgb__gamma': None, 'xgb__gpu_id': None, 'xgb__grow_policy': None, 'xgb__importance_type': None, 'xgb__interaction_constraints': None, 'xgb__learning_rate': 0.1, 'xgb__max_bin': None, 'xgb__max_cat_threshold': None, 'xgb__max_cat_to_onehot': None, 'xgb__max_delta_step': None, 'xgb__max_depth': None, 'xgb__max_leaves': None, 'xgb__min_child_weight': None, 'xgb__missing': nan, 'xgb__monotone_constraints': None, 'xgb__n_estimators': 500, 'xgb__n_jobs': None, 'xgb__num_parallel_tree': None, 'xgb__predictor': None, 'xgb__random_state': 31510, 'xgb__reg_alpha': None, 'xgb__reg_lambda': None, 'xgb__sampling_method': None, 'xgb__scale_pos_weight': None, 'xgb__subsample': None, 'xgb__tree_method': None, 'xgb__validate_parameters': None, 'xgb__verbosity': None, 'lgb__boosting_type': 'dart', 'lgb__class_weight': None, 'lgb__colsample_bytree': 1.0, 'lgb__importance_type': 'split', 'lgb__learning_rate': 0.06, 'lgb__max_depth': 5, 'lgb__min_child_samples': 20, 'lgb__min_child_weight': 0.001, 'lgb__min_split_gain': 0.0, 'lgb__n_estimators': 100, 'lgb__n_jobs': -1, 'lgb__num_leaves': 31, 'lgb__objective': None, 'lgb__random_state': 17228, 'lgb__reg_alpha': 0.0, 'lgb__reg_lambda': 0.0, 'lgb__subsample': 1.0, 'lgb__subsample_for_bin': 200000, 'lgb__subsample_freq': 0, 'lgb__verbosity': -1, 'cat__learning_rate': 0.01, 'cat__depth': 3, 'cat__loss_function': 'RMSE', 'cat__verbose': 0, 'cat__num_boost_round': 100, 'cat__random_state': 28160}``

In [ ]:
# # Guardamos el modelo Votación
# dump(clf, 'clasificador.joblib')
# dump(reg, 'regresor.joblib')

---

## **5. Evaluación y Selección del Modelo**

In [ ]:
data = df.copy()

In [ ]:
X = data.drop(columns=['transactionRevenue'])
y = data['transactionRevenue']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
# Calcular la media
mean_value = df['transactionRevenue'].mean()

# Crear un array con la media repetida para cada observación
mean_array = np.full(shape=df['transactionRevenue'].shape, fill_value=mean_value)

# Calcular el MSE
mse = mean_squared_error(df['transactionRevenue'], mean_array)

print(f'La raiz del error cuadrático para la media: {np.sqrt(mse):.3f}')

In [ ]:
# Comparar modelos de Regresión gráficamente
plt.figure(figsize=(10, 6))
plt.scatter(range(len(y_test)), y_test, color='blue', label='Real', alpha=0.5)
plt.scatter(range(len(reg_xgb_pred)), reg_xgb_pred, color='red', label='Predicho XGB', alpha=0.5)
plt.scatter(range(len(reg_lgb_pred)), reg_lgb_pred, color='green', label='Predicho LGB', alpha=0.5)
plt.scatter(range(len(reg_cat_pred)), reg_cat_pred, color='orange', label='Predicho CAT', alpha=0.5)
plt.scatter(range(len(y_pred_reg)), y_pred_reg, color='purple', label='Predicho Votación', alpha=0.5)
plt.title('Comparación de Modelos de Regresión')
plt.xlabel('Índice')
plt.ylabel('transactionRevenue')
plt.legend()
plt.show()

In [ ]:
# Comparar modelos de Regresion con el rmse gráficamente mediante barras
rmse = [xgb_rmse, lgb_rmse, cat_rmse, vot_rmse]
modelos = ['XGB', 'LGB', 'CAT', 'Votación']
plt.figure(figsize=(10, 6))
bars = plt.bar(modelos, rmse, color=['red', 'green', 'orange', 'purple'])
plt.title('Comparación de Modelos de Regresión')
plt.xlabel('Modelo')
plt.ylabel('Raiz del Error')

# Añadir el valor de cada barra en la gráfica
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 2), va='bottom')  # va: vertical alignment

# R2 = 0
plt.axhline(y=np.round(np.sqrt(mse), 2), color='r', linestyle='--')
# Metrica de Alejo
plt.axhline(y=16.76, color='b', linestyle='solid')

plt.show()